We will use a new package ``mlxtend`` for association rules mining. Check the [documentation](https://rasbt.github.io/mlxtend/) for details.  

You may need to install the package first with the following code. 

```python
pip install mlxtend
```

# 1. Data Preprocessing

We sampled 9958 Tweets with two or more food/drink emojis, each tweets can be considered as a itemset for emojis.   

- Let's start by loading the dataset and inspect its first few lines.

In [2]:
import pandas as pd

tweets_df = pd.read_csv("food_drink_emoji_tweets.txt", sep="\t", header=None)  # tab separated data, ignore header

tweets_df.columns = ['text']     # rename the column name  

tweets_df.head()

,text
0,RT @CalorieFixess: 🍗🌯🍔🍒 400 Calories https://t...
1,RT @1_F_I_R_S_T: _ 🍈¹〉Grow your account fast! ...
2,RT @LegendDeols: 👉👉👉G€T Ready to dance💃🕺🕺🕺💃💃💃 ...
3,@britch_x Hubby's friend bought us Wendy's-che...
4,RT @DAILYPUPPlES: Workout partner ☕🍌😍 https://...


### 1.1 Extract food/drink emojis in each tweet

Each row is a tweet/post. Let's extract the emojis that appear in each post as an itemset. 

- We are only interested in emojis that are food and drinks.

**Step 1: create a emoji set**

In [3]:
emoji_list = "🍇🍈🍉🍊🍋🍌🍍🥭🍎🍏🍐🍑🍒🍓🥝🍅🥥🥑🍆🥔🥕🌽🌶🥒🥬🥦🍄🥜🌰🍞🥐🥖🥨🥯🥞🧀🍖🍗🥩🥓🍔🍟🍕🌭🥪🌮🌯🥙🥚🍳🥘🍲🥣🥗🍿🧂🥫🍱🍘🍙🍚🍛🍜🍝🍠🍢🍣🍤🍥🥮🍡🥟🥠🥡🦀🦞🦐🦑🍦🍧🍨🍩🍪🎂🍰🧁🥧🍫🍬🍭🍮🍯🍼🥛☕🍵🍶🍾🍷🍸🍹🍺🍻🥂🥃"
        
emoji_set = set(emoji_list)  # convert the sequence of emjois as itemset

#emoji_set    # uncomment to see the result

**Step 2: extract emoji from each tweet as a set**

In [4]:
def extract_unique_emoji(text):
    emoji_list_per_tweet = [emoji for emoji in text if emoji in emoji_set]       # list comphrehension
    emoji_set_per_tweet = set(emoji_list_per_tweet)                              # convert the list as a set 
    return  emoji_set_per_tweet                                        

tweets_df['emojis'] = tweets_df['text'].apply(extract_unique_emoji)        # apply the function to column 'text'
tweets_df.head()

,text,emojis
0,RT @CalorieFixess: 🍗🌯🍔🍒 400 Calories https://t...,"{🍒, 🌯, 🍗, 🍔}"
1,RT @1_F_I_R_S_T: _ 🍈¹〉Grow your account fast! ...,"{🍊, 🍓, 🍉, 🍈, 🍇, 🍍}"
2,RT @LegendDeols: 👉👉👉G€T Ready to dance💃🕺🕺🕺💃💃💃 ...,"{🥃, 🍸}"
3,@britch_x Hubby's friend bought us Wendy's-che...,"{🍟, 🍔}"
4,RT @DAILYPUPPlES: Workout partner ☕🍌😍 https://...,"{🍌, ☕}"


### 1.2 Transform emoji items into matrix 

Before we use the `mlxtend` for frequent itemset mining, we need to encode the items into a matrix where (1) each row represents an itemset (tweet) and (2) each column represents an item, and (3) each cell record whether an item is in the itemset (tweet) or not.  

- Let's use the `TransactionEncoder` function from `mlxtend.preprocessing`. Check [documentation](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.preprocessing/#transactionencoder) for details.
- Alternatively, you may also use the `MultiLabelBinarizer` from `scikit-learn`. Check [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html) for details.

In [5]:
from mlxtend.preprocessing import TransactionEncoder

encoder = TransactionEncoder()

arr = encoder.fit_transform(tweets_df['emojis'])   # return a matrix

arr   # a 2D array of boolean values 

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], shape=(9958, 105))

In [6]:
# convert as dataframe

emoji_df = pd.DataFrame(data = arr,  columns = encoder.columns_)

emoji_df

,☕,🌭,🌮,🌯,🌰,🌶,🌽,🍄,🍅,🍆,...,🥭,🥮,🥯,🦀,🦐,🦑,🦞,🧀,🧁,🧂
0,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9953,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9954,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9955,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9956,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# 2.  Frequent Itemsets Mining

## 2.1 Find frequent itemsets 

Now, let's apply the Apriori Algorithm to the emoji dataset.

Call the `apriori` API from `mlxtend.frequent_patterns` and specify the minimal support we want. Check the [User Guide](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) and [Documentation](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/#apriori) for more details.

- set `min_support=0.005`, i.e., half percent of the tweets. 
- set ``use_colnames=True`` to convert column indices of each item (default) into the respective item names for readability.

In [7]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df = emoji_df, min_support = 0.005, use_colnames = True)

frequent_itemsets

,support,itemsets
0,0.058747,(☕)
1,0.019381,(🌭)
2,0.018678,(🌮)
3,0.008536,(🌯)
4,0.005322,(🌰)
...,...,...
260,0.005724,"(🍸, 🍷, 🍺, 🍹)"
261,0.006126,"(🍸, 🍷, 🍻, 🍹)"
262,0.005624,"(🍸, 🥂, 🍷, 🍹)"
263,0.005021,"(🍸, 🍷, 🍺, 🍻)"


## 2.2 Sort frequent itemsets by their support values

In [8]:
frequent_itemsets.sort_values(by = 'support',ascending = False)

,support,itemsets
55,0.181663,(🍻)
59,0.148725,(🎂)
22,0.138783,(🍔)
57,0.108255,(🍾)
60,0.103234,(🥂)
...,...,...
230,0.005021,"(🥃, 🍷, 🍹)"
233,0.005021,"(🍷, 🍾, 🍻)"
258,0.005021,"(🍊, 🍇, 🥝, 🍍)"
257,0.005021,"(🍇, 🍉, 🥝, 🍍)"


<font color=red>***Exercise 1: Your Codes Here***</font>  

**Step 1**: How many itemsets appears in at least 10% of the posts?

In [ ]:
high_support = frequent_itemsets[frequent_itemsets['support'] >= 0.10]
high_support.shape[0]

**Step 2**: Extract frequent itemsets with at least two 2 items.

- Hint: use `pandas.Series.apply` function together with the `len` to check the number of items in each itemset first.

In [ ]:
frequent_itemsets_2plus = frequent_itemsets[frequent_itemsets['itemsets'].apply(len) >= 2]
frequent_itemsets_2plus

# 3.  Association Rules 

In this section, we will practice how to evaluate the relationship between frequent itemsets. 

- Let's call the `association_rules` API from `mlxtend.frequent_patterns`. Visit the [User Guide](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/)  and [Documentation](https://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/#association_rules) for more details.

The association rules represent the two relationship between to frequent itemsets: an `antecedent` item(set) and a `consequent` item(set). 

In [9]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(df = frequent_itemsets,     # frequent itemsets generated by apriori algorihm
                          metric = "lift",         
                          min_threshold = 1)          # min lift = 1

rules.sort_values(by = 'lift',ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
559,"(🍊, 🥝)","(🍍, 🍇)",0.007030,0.007833,0.005021,0.714286,91.190476,1.0,0.004966,3.472585,0.996036,0.510204,0.712030,0.677656
562,"(🍍, 🍇)","(🍊, 🥝)",0.007833,0.007030,0.005021,0.641026,91.190476,1.0,0.004966,2.766132,0.996842,0.510204,0.638484,0.677656
575,"(🍉, 🥝)","(🍊, 🍍)",0.007833,0.007833,0.005222,0.666667,85.111111,1.0,0.005161,2.976501,0.996053,0.500000,0.664035,0.666667
574,"(🍊, 🍍)","(🍉, 🥝)",0.007833,0.007833,0.005222,0.666667,85.111111,1.0,0.005161,2.976501,0.996053,0.500000,0.664035,0.666667
560,"(🍊, 🍍)","(🥝, 🍇)",0.007833,0.007632,0.005021,0.641026,83.991228,1.0,0.004961,2.764454,0.995895,0.480769,0.638265,0.649460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,(🍉),(🎂),0.055232,0.148725,0.009239,0.167273,1.124714,1.0,0.001024,1.022274,0.117368,0.047447,0.021789,0.114696
173,(🍻),(🍷),0.181663,0.072705,0.014461,0.079602,1.094857,1.0,0.001253,1.007493,0.105872,0.060276,0.007437,0.139249
172,(🍷),(🍻),0.072705,0.181663,0.014461,0.198895,1.094857,1.0,0.001253,1.021510,0.093432,0.060276,0.021057,0.139249
160,(🎂),(🍭),0.148725,0.049307,0.007732,0.051992,1.054451,1.0,0.000399,1.002832,0.060661,0.040633,0.002824,0.104407


In the returned data frame, each row examines a pair of `antecedent` (X) -> `consequent` (Y).

- While the `antecedent support` and `consequent support` measure $P(X)$ and $P(Y)$ respectively, the `support` measures the joint probability $P(X, Y)`.

Other metrics such as `confidence`, `lift`, `leverage`, and `conviction` can be derived from the three support values. For example,

$$\text{confidence} = rac{\text{support}}{\text{antecedent_support}}$$

$$\text{lift} = rac{\text{confidence}}{\text{consequent_support}} = rac{\text{support}}{\text{antecedent_support} \times \text{consequent_support}}$$

$$\text{leverage} = \text{support} - \text{antecedent_support} \times \text{consequent_support}$$


# 4.  Itemset Similarity

Pattern and similarity are two basic outputs of data mining. So far, we have been playing with patterns - frequent itemsets and association rules can all be seen as "patterns". In the last part, let's work on itemset similarities.
 
**Jaccard similarity** is a simple but powerful measurement of itemset similarity, defined as follows:
$$\text{Jaccard\_similarity(A, B)} = \frac{|A\cap B|}{|A\cup B|}$$



In [10]:
def jaccard_similarity(A, B):
    intersection = A.intersection(B)
    union = A.union(B)
    jaccard_sim = len(intersection)/len(union)
    return jaccard_sim

Calculate the jaccard similarity of two sets.

In [11]:
fruit_A = set(['🍇', '🍒', '🍎'])
fruit_B = set(['🍊', '🍒', '🍎'])

jaccard_similarity(fruit_A, fruit_B)

0.5

**Which tweet is most similiar to the first tweet post in terms of the set of food and drink emojis used?** 


**Step 1**: define a function which returns the jaccard similarity between the first itemset and any itemsets.

In [12]:
def jaccard_one(X):
    first_tweet = tweets_df.loc[0, 'emojis']
    jaccard_sim = jaccard_similarity(first_tweet, X)
    return jaccard_sim

<font color=red>***Exercise 2: Your Codes Here***</font>  

**Step 2**: calculate the jaccard similarity of the first emoji itemset and the itemsets in each row, and save the jaccard similarities a separate column named `jaccard`. 

In [ ]:
tweets_df['jaccard'] = tweets_df['emojis'].apply(jaccard_one)

tweets_df[['emojis', 'jaccard']].head()

**Step 3**: sort the dataframe by the column `jaccard` and check the top 10 rows.

In [ ]:
tweets_df.sort_values(by='jaccard', ascending=False).head(10)